In [ ]:
# import xlsxwriter
import openpyxl
from openpyxl.styles.fonts import Font
from openpyxl.styles.borders import Border, Side
import mysql.connector


def fetch_table_data(table_name):
    conx = None
    try:
        # The connect() constructor creates a connection to the MySQL server and returns a MySQLConnection object.
        cnx = mysql.connector.connect(
            host='db',
            database='globis-dev',
            user='root',
            password='root'
        )

        cnx.ping(reconnect=True)
        print(cnx.is_connected())

        cursor = cnx.cursor()
        cursor.execute('select * from ' + table_name)

        header = [row[0] for row in cursor.description]
        rows = cursor.fetchall()

        # Closing connection
        cnx.close()

    except Exception as e:
        print("Error Occurred: {e}")
    finally:
        if cnx is not None and cnx.is_connected():
            cnx.close()

    return header, rows


def export(table_name):
    # Create an new Excel file and add a worksheet.
    wb = openpyxl.Workbook()
    ws = wb.worksheets[0]
    ws.title = 'Menu'
    # wb = openpyxl.load_workbook(filename=table_name + '.xlsx')
    # sheets = wb.sheetnames
    # print(sheets)

    # Create style for cells
    header_cell_format = Font(name = 'メイリオ', size = 16, bold = True, color = '000000ff')
    body_cell_format = Border(top = Side(style = 'thin', color = '000000'),
        bottom = Side(style = 'thin', color = '000000'),
        left = Side(style = 'thin', color = '000000'),
        right = Side(style = 'thin', color = '000000')
    )
    header, rows = fetch_table_data(table_name)

    row_index = 1
    column_index = 1
    # header
    # for col in ws.columns:
    for c in header:
        cell1 = ws.cell(row = 1, column = column_index)
        cell1.font = header_cell_format
        cell1.border = body_cell_format
        # ws.cell(row=1, column=column_index, value=header[column_index-1])
        cell1.value = header[column_index-1]
        column_index += 1

    row_index += 1
    for row in rows:
        column_index = 1
        for column in row:
            cell2 = ws.cell(row = row_index, column = column_index)
            cell2.font = header_cell_format
            cell2.border = body_cell_format
            cell2.value = column

            column_index += 1
        row_index += 1

    print(str(row_index) + ' rows written successfully to ' + table_name + '.xlsx')

    wb.save(filename=table_name + '.xlsx')
    wb.close()

# Tables to be exported
export('maintenances')
# export('TABLE_2')

: 